# Projeto AI/ML Rumos AI Academy

## Conjunto de dados

Este conjunto de dados contém os links de phishing
O dataset foi recolhido [aqui](https://archive.ics.uci.edu/dataset/967/phiusiil+phishing+url+dataset)

In [25]:
import pandas as pd
import matplotlib.pyplot as plt


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split

# read only 100 lines for now
df = pd.read_csv('./geospatial_data.csv', nrows=100)

In [ ]:
# let's inspect the data
print(df.head)

In [ ]:
# plot the data

dependent_var = 'accident_risk'

# List of columns to plot accident_risk against, excluding non-numeric ones
independent_vars = ['air_quality_index', 'temperature', 'humidity', 'traffic_density', 
                    'average_speed', 'population_density', 'noise_levels_db', 'building_density', 'congestion_level']


# Create subplots for each independent variable
plt.figure(figsize=(15, 20))
for i, col in enumerate(independent_vars):
    plt.subplot(len(independent_vars) // 2 + 1, 2, i + 1)  # Arrange in grid
    plt.scatter(df[col], df[dependent_var], alpha=0.7, label=f"{col} vs {dependent_var}")
    plt.xlabel(col)
    plt.ylabel(dependent_var)
    plt.title(f"{dependent_var} vs {col}")
    plt.grid(True)
    plt.legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [38]:
# solve the encoding problem of the categorical variables
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['vehicle_type_encoded'] = encoder.fit_transform(df['vehicle_type'])
df['favorite_color_encoded'] = encoder.fit_transform(df['favorite_color'])
df['ice_cream_preference_encoded'] = encoder.fit_transform(df['ice_cream_preference'])

# change the timestamp data, which is a string and needs to be numeric values
df['timestamp_converted'] = pd.to_datetime(df['timestamp'])

In [51]:
# dividir os dados em variáveis de entrada e alvo

X = df[['latitude','longitude','air_quality_index','temperature','humidity','traffic_density','vehicle_type_encoded','average_speed','population_density','noise_levels_db','building_density','congestion_level','favorite_color_encoded','ice_cream_preference_encoded']]
y = df['accident_risk']

In [ ]:
print(y)

In [ ]:
print(X)

In [ ]:
# dividir em conjunto de treino e temporário
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4,random_state=42)

# dividir em conjunto de validação e teste - 50% dos 60% que sobraram da linha anterior - logo 30% para teste e 30% para validação
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5,random_state=42)


### Regressão Linear Simples

In [53]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

In [56]:
# Regressão Linear Simples
# Só usamos um dos valores de x (uma das variáveis independentes)
lr_simple = LinearRegression()

# neste caso vamos só usar a temperatura para treinar o nosso modelo - lr_simple
lr_simple.fit(X_train[['traffic_density']], y_train)

# vamos agora passar o array com as features de validação e verificar se o modelo prevê corretamente
y_val_pred_simple = lr_simple.predict(X_val[['traffic_density']])

# vamos agora comparar os valores de y recebidos com os x_val com os y_val que já conhecíamos dos dados
r2_simple = r2_score(y_val, y_val_pred_simple)

mae_simple = mean_absolute_error(y_val, y_val_pred_simple)

model_performance  = {'Regressao_Linear_Simples': (lr_simple, r2_simple,mae_simple)}


### Regressão Linear Multipla

In [ ]:
print(df.dtypes)

In [58]:
lr_multiple = LinearRegression()

# tran the model with all x (independent variables)
lr_multiple.fit(X_train, y_train)

LinearRegression()

In [72]:
# predict with validation set
y_val_pred_multiple = lr_multiple.predict(X_val)

# get metrics for validation group
r2_multiple = r2_score(y_val, y_val_pred_multiple)
mae_multiple = mean_absolute_error(y_val, y_val_pred_multiple)

model_performance['Regressao_Linear_Multipla_val'] = (lr_multiple, r2_multiple, mae_multiple)

# predict with test set
y_test_pred_multiple = lr_multiple.predict(X_test)

# get metrics fro testing group
r2_multiple_test = r2_score(y_test, y_test_pred_multiple)
mae_multiple_test = mean_absolute_error(y_test, y_test_pred_multiple)

model_performance['Regressao_Linear_Multipla_test'] = (lr_multiple, r2_multiple_test, mae_multiple_test)

In [73]:
from sklearn.linear_model import Lasso

# regressão lasso
lasso = Lasso(alpha=0.1)

# treinar o modelo
lasso.fit(X_train, y_train)

# comparar com o grupo de validação
y_val_pred_lasso = lasso.predict(X_val)

#get metrics for validation group
r2_lasso = r2_score(y_val, y_val_pred_lasso)
mae_lasso = mean_absolute_error(y_val, y_val_pred_lasso)

model_performance['Regressao_Lasso_val'] = (lasso, r2_lasso, mae_lasso)

# predict using test data set
y_test_pred_lasso = lasso.predict(X_test)

# get metrics for test group
r2_lasso_test = r2_score(y_test, y_test_pred_lasso)
mae_lasso_test = mean_absolute_error(y_test, y_test_pred_lasso)

model_performance['Regressao_Lasso_test'] = (lasso, r2_lasso_test, mae_lasso_test)



In [75]:
### Cell to compare models performance

for item in model_performance:
    value = model_performance[item]
    print(item, value)




Regressao_Linear_Simples (LinearRegression(), 0.5730895614757552, np.float64(0.06257161600375381))
Regressao_Linear_Multipla (LinearRegression(), 0.7858669891737105, np.float64(0.04293508930633331))
Regressao_Linear_Multipla_val (LinearRegression(), 0.7858669891737105, np.float64(0.04293508930633331))
Regressao_Linear_Multipla_test (LinearRegression(), 0.8660270824525497, np.float64(0.052643030047848274))
Regressao_Lasso_val (Lasso(alpha=0.1), 0.6963524397085528, np.float64(0.05130244333610545))
Regressao_Lasso_test (Lasso(alpha=0.1), 0.8639832180075667, np.float64(0.047945711187428366))


In [76]:
# select best model - greater r2
# Selecionar o melhor modelo com base no R² de validação
melhor_nome, (melhor_modelo, melhor_r2, melhor_mae) = max(model_performance.items(), key=lambda x: x[1][1])
print(f"Melhor modelo baseado no R² de validação: {melhor_nome}")

Melhor modelo baseado no R² de validação: Regressao_Linear_Multipla_test
